# DiracPy Tutorial

In [1]:
import diracpy as dp
import numpy as np

## A Brief Overview

Diracpy is a python package that allows one to rapdily build and execute quantum models by coding them directly in a syntax that resembles dirac notation. For example, to define the Hamiltonian
$$
    H_0 = \hbar \omega a^{\dagger}a
$$
where $a$ and $a^{\dagger}$ are bosonic creation and annihilation operators associated with a cavity mode
we can right code such as
```
    H_0 = hbar * omega * cav.adag * cav.a
```
which obeys exactly the same mathematical operations as one would expect. Suppose that we the action of this state vector such as $\vert n \rangle$, i.e.
$$
    H_0 \vert \psi \rangle = \hbar \omega a^{\dagger} a \vert n \rangle,
$$
then we simply define the Fock state $\vert n \rangle$ using ```dp.ket(n)``` with variable ```n``` and act on this with the ```H_0``` we defined above, i.e.
```
    H_0 * dp.ket(n)
```

All the rules of state vectors (ket's), dual vectors (bra's) and linear operators are encoded into diracpy. This allows one to define operators abstractly without rather than dealing directly with matrices which require a predefined basis. The advantage is that the sytem can be coded in a very straightforward manner. For example ...

When we write a state vector, we inherently decide what the indices mean, and how operators act on such state vectors. For example, the we know that the Fock state $\vert n \rangle$ is represented by the photon number $n$ and that the creation and annihlation operators have the actions $a \vert n \rangle = \sqrt{n} \vert n-1 \rangle$, $a^\dagger \vert n \rangle = \sqrt{n+1} \vert n+1 \rangle$.

In diracpy, we define a the fock space for example by creating the fock space object
```
    cav1 = dp.fock_subspace(index = 0)
```
which asigns the first index of any subsequent state vector to this Fock space, which we have called cav1. Thus we could now define the single photon ket $\vert 1 \rangle$ using
```
    dp.ket([1])
```

In [11]:
cav1 = dp.fock_subspace()
psi = dp.ket([1])
print(psi)

ket[1]


In defining the fock space ```cav1``` we have also defined the accompanying operators $a$ and $a^{\dagger}$ which we access using ```cav1.a``` and ```cav1.adag```. 

In [12]:
print (cav1.adag * psi)
print (cav.a * psi)

1.4142135623730951 * ket[2]
ket[0]


This example highlights how we are able to define state vectors and state spaces without explicitly giving a list of basis states.

Suppose we want to define a second cavity mode. The state space then has two Fock subspaces, and we would write states in the form $\vert n, m \rangle$ where we have assigned the first index to the first mode and the second index to the second mode. In diracpy we would define such as system as follows

Still to cover in overview:

- Quantum systems
- Dynamics

## qvec's

There are two types of qvec (short for quantum vector) objects used in diracpy, bra's and ket's. These both inherit from the private class qvec that defines some fundamental shared properties. The ket qvec's span the Hilbert space of a quantum system, while bra's span the dual space. In the simplest case, qvec objects are instantiated with a list of indexes that define the quantum state, together with a complex coefficient which defaults to one. 

In the default case of unity coefficient, the qvecs are therefore normalised to unity. For example, we could define the orthonormal basis states of a two level system as $\{\vert g \rangle, \vert e \rangle \}$.

In [2]:
down = dp.ket(['g'])
up = dp.ket(['e'])

Another property of qvecs is that a superposition of qvecs is also a qvec. So we could define the state
$$
    \vert \psi \rangle = \frac{1}{\sqrt 2} \left( \vert g \rangle + \vert e \rangle \right)
$$
as

In [3]:
psi = 1/np.sqrt(2) * (down + up)
type(psi)
print(psi)

0.7071067811865475 * ket['g'] + 0.7071067811865475 * ket['e']


Here we see that psi is also a ket (qvec) object. We also see in this example that a ket multiplied by a scalar is also a ket. 

In fact, kets (and qvecs in general) satisfy all axioms of vector spaces. Associativity and addi

In [4]:
u, v, w = dp.ket(['u']), dp.ket(['v']), dp.ket(['w'])
zeroket = dp.ket()
print((u + v) + w == u + (v + w))
print(u + v == v + u)
print(u + zeroket == u)
print(u + (-u) == zeroket)

True
True
True
True


Under scalar multiplication we inherit the associative and commutative properties of scalars. The ramaining vector space axioms are also satisfied. E.g.

In [13]:
a, b = 2, 3
print(1 * u == u)
print((a + b) * u == a * u + b * u)
print(a * (u + v) == a * u + a * v)

True
True
True


All of the above is true for bra qvecs, which we could define explicitly, e.g ```dp.bra(['e'])```, or we could use the conjugate method of any qvec:

In [29]:
type(down.conj())

diracpy.states_operators.bra

In [27]:
psi_star = 1/np.sqrt(2) * (down.conj() + up.conj())
type(psi_star)
print(psi_star)

0.7071067811865475 * bra['g'] + 0.7071067811865475 * bra['e']


or most simply,

In [28]:
psi_star = psi.conj()
type(psi_star)
print(psi_star)

0.7071067811865475 * bra['g'] + 0.7071067811865475 * bra['e']


Inner product...

Finally, we can define operators using the outer product of a ket and bra. For example we could define the raising operator $\vert e \rangle \langle g \vert$ as

In [32]:
raise_op = up * down.conj()
type(raise_op)

which is no longer a `bra` or `ket` type object, but a `qop` object which can act to the right on kets or to the left on bras, where for example we expect $\vert e \rangle \langle g \vert g \rangle = \vert e \rangle $, and $\langle e \vert e \rangle \langle g \vert = \langle g \vert$.

In [35]:
print(raise_op * down)
print(up.conj() * raise_op)

ket['e']
bra['g']


We will consider more general ways to define operators in the state space section below.

## State Spaces

Thus far we have only considered quantum systems that can be defined in terms of bra's and ket's together with their outer products. This is sufficient for small state spaces where we maybe able to define all the neccesary operators of the Hamiltonian explicitly. However, mathematically speaking, for larger state spaces such a limited notation is not practical. Instead, we combine the dirac vectors and dual vectors with linear operators, which we can define in a more abstract scence. A good example of this are the bosonic raising or lowering operators on Fock states where
$$
    a^\dagger \vert n \rangle = \sqrt{n+1} \vert n+1 \rangle,\quad
    a \vert n \rangle = \sqrt{n} \vert n-1 \rangle
$$
which are true given any $n\geq0$.
These would be impractical to define using the outer products, e.g. for $a^\dagger$ we could need the infinite sum $a^\dagger = \vert 1 \rangle \langle 0 \vert + \sqrt{2} \vert 2 \rangle \langle 1 \vert + \dots + \sqrt{n+1}\vert n+1 \rangle \langle n \vert + \dots$, particulary as we know the general pattern given any $n$.

Diracpy allows ony to define operators abstractly wherever there exists a well defined mapping of indices and coefficients. For example, $a^\dagger$ maps the index $n$ onto $n+1$, and the Fock states coefficient $c$ onto $c \sqrt {n+1}$.